In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
# di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

import hail as hl
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://633fc3f91d0f:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.29-cf730c8fc8f6
LOGGING: writing to /hail/large_vcf_filtering_annotation/hail-20191221-0735-0.2.29-cf730c8fc8f6.log


In [2]:
rpmk_files = !ls ./repeatmasker_split

In [4]:
for file in rpmk_files:
    hl.import_bed('./repeatmasker_split/'+file, reference_genome='GRCh38', skip_invalid_intervals=True).write(file+'.ht')

2019-12-21 07:36:14 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (user-specified)
  Loading column 'f1' as type 'int32' (user-specified)
  Loading column 'f2' as type 'int32' (user-specified)
  Loading column 'f3' as type 'str' (user-specified)
  Loading column 'f4' as type 'str' (user-specified)
  Loading column 'f5' as type 'str' (type not specified)

2019-12-21 07:36:16 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-21 07:36:18 Hail: INFO: wrote table with 200000 rows in 12 partitions to rpmsk00.ht
2019-12-21 07:36:18 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (user-specified)
  Loading column 'f1' as type 'int32' (user-specified)
  Loading column 'f2' as type 'int32' (user-specified)
  Loading column 'f3' as type 'str' (user-specified)
  Loading column 'f4' as type 'str' (user-specified)
  Loading column 'f5' as type 'str' (type not specified)

2019-12-21 07:36:18 Hail: INFO: Ord

2019-12-21 07:36:32 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-21 07:36:32 Hail: INFO: wrote table with 200000 rows in 12 partitions to rpmsk14.ht
2019-12-21 07:36:32 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (user-specified)
  Loading column 'f1' as type 'int32' (user-specified)
  Loading column 'f2' as type 'int32' (user-specified)
  Loading column 'f3' as type 'str' (user-specified)
  Loading column 'f4' as type 'str' (user-specified)
  Loading column 'f5' as type 'str' (type not specified)

2019-12-21 07:36:33 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-21 07:36:33 Hail: INFO: wrote table with 200000 rows in 12 partitions to rpmsk15.ht
2019-12-21 07:36:33 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (user-specified)
  Loading column 'f1' as type 'int32' (user-specified)
  Loading column 'f2' as type 'int32' (user-specified)
  Loading column 'f3' as typ

In [18]:
gnmd = hl.read_table('/hail/gnomad/gnomad.genomes.r3.0.sites.ht')

In [19]:
gnmd.count()

707950943

In [20]:
for file in rpmk_files:
    rpmk = hl.read_table(file+'.ht')
    gnmd = gnmd.filter(hl.is_defined(rpmk[gnmd.locus]), keep = False)
    gnmd = gnmd.key_by()
    gnmd = gnmd.key_by(gnmd.locus, gnmd.alleles)

In [21]:
gnmd = gnmd.select(v3_nfe = gnmd.freq[2], 
            intergenic_consequences = gnmd.vep.intergenic_consequences,
            most_severe_consequence = gnmd.vep.most_severe_consequence,
            motif_feature_consequences = gnmd.vep.motif_feature_consequences,
            regulatory_feature_consequences = gnmd.vep.regulatory_feature_consequences,
            transcript_consequences = gnmd.vep.transcript_consequences,
            variant_class = gnmd.vep.variant_class,
            rsid = gnmd.rsid)

In [22]:
cov = hl.read_table('/hail/gnomad/gnomad.genomes.r3.0.coverage.ht')

In [23]:
cov = cov.select(cov.over_1)

In [24]:
gnmd = gnmd.annotate(cov_v3 = cov[gnmd.locus])

In [25]:
gnmd.describe()

----------------------------------------
Global fields:
    'freq_meta': array<dict<str, str>> 
    'filtering_model': struct {
        model_id: str, 
        model_name: str, 
        score_name: str, 
        snv_cutoff: struct {
            bin: int32, 
            min_score: float64
        }, 
        indel_cutoff: struct {
            bin: int32, 
            min_score: float64
        }
    } 
    'freq_index_dict': dict<str, int32> 
    'faf_index_dict': dict<str, int32> 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'v3_nfe': struct {
        AC: int32, 
        AF: float64, 
        AN: int32, 
        homozygote_count: int32
    } 
    'intergenic_consequences': array<struct {
        allele_num: int32, 
        consequence_terms: array<str>, 
        impact: str, 
        minimised: int32, 
        variant_allele: str
    }> 
    'most_severe_consequence': str 
    'motif_feature_consequences': array<struct 

In [26]:
gnmd = gnmd.filter(gnmd.cov_v3.over_1 > 0.9)

In [ ]:
gnmd.write('/hail/gnomad/gnomad_selected_filtered.ht')

In [28]:
gnmd = hl.read_table('/hail/gnomad/gnomad_selected_filtered.ht')

In [33]:
gnmd.count()

296269085

In [34]:
hl.import_table('./cadd/whole_genome_SNVs.tsv.gz', force_bgz = True, delimiter='\t', comment='#', 
                     types = {'f0':'tstr', 'f1':'tint', 'f2':'tstr', 'f3':'tstr', 'f4':'tfloat', 'f5':'tfloat'},
                     no_header = True).write('./cadd/cadd.ht')

2019-12-22 16:42:56 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (user-specified)
  Loading column 'f1' as type 'int32' (user-specified)
  Loading column 'f2' as type 'str' (user-specified)
  Loading column 'f3' as type 'str' (user-specified)
  Loading column 'f4' as type 'float64' (user-specified)
  Loading column 'f5' as type 'float64' (user-specified)

2019-12-22 17:40:45 Hail: INFO: wrote table with 8812917339 rows in 2558 partitions to ./cadd/cadd.ht


In [35]:
cadd = hl.read_table('./cadd/cadd.ht')

In [37]:
cadd.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'f0': str 
    'f1': int32 
    'f2': str 
    'f3': str 
    'f4': float64 
    'f5': float64 
----------------------------------------
Key: []
----------------------------------------


In [40]:
cadd = cadd.select(variants = hl.parse_variant('chr'+hl.str(cadd.f0)+':'+hl.str(cadd.f1)+':'+hl.str(cadd.f2)+':'+hl.str(cadd.f3), reference_genome='GRCh38'),
               cadd_score = cadd.f5)

In [42]:
cadd = cadd.key_by(cadd.variants.locus,cadd.variants.alleles)

In [45]:
gnmd = gnmd.annotate(cadd = cadd[gnmd.key])
gnmd = gnmd.transmute(cadd = gnmd.cadd.cadd_score)

In [ ]:
gnmd.checkpoint('/hail/gnomad/gnomad_selected_filtered_cadd.ht')

2019-12-22 19:16:29 Hail: INFO: Ordering unsorted dataset with network shuffle


In [48]:
gnmd.count()

296269085